In [1]:
from sklearn.datasets import load_iris, fetch_20newsgroups
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

In [2]:
# 添加数据集
iris_dataset = load_iris()
X = iris_dataset['data']
Y = iris_dataset['target']

In [3]:
# 数据划分
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=0)
print(Y_train)
print('*' * 50)
print(Y_test)

[1 1 2 0 2 0 0 1 2 2 2 2 1 2 1 1 2 2 2 2 1 2 1 0 2 1 1 1 1 2 0 0 2 1 0 0 1
 0 2 1 0 1 2 1 0 2 2 2 2 0 0 2 2 0 2 0 2 2 0 0 2 0 0 0 1 2 2 0 0 0 1 1 0 0
 1 0 2 1 2 1 0 2 0 2 0 0 2 0 2 1 1 1 2 2 1 1 0 1 2 2 0 1 1 1 1 0 0 0 2 1 2
 0]
**************************************************
[2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0 2 1 0 2 2 1 0
 1]


In [4]:
# 标准化
# 特征工程（标准化）,下面3行注释，一开始我们不进行标准化，看下效果，目标值要不要标准化？
std = StandardScaler()
# #
# # # 对测试集和训练集的特征值进行标准化,服务于knn fit
X_train = std.fit_transform(X_train)
# # transform返回的是copy，不在原有的输入对象中去修改
# print(id(x_test))
X_test = std.transform(X_test)  #transfrom不再进行均值和方差的计算，是在原有的基础上去标准化

In [5]:
knn = KNeighborsClassifier()

In [6]:
# # 构造一些参数的值进行搜索
param = {"n_neighbors": [i for i in range(1, 11)]}
#
# 进行网格搜索，cv=3是3折交叉验证，用其中2折训练，1折验证
gc = GridSearchCV(knn, param_grid=param, cv=3)

gc.fit(X_train, Y_train)  #你给它的x_train，它又分为训练集，验证集

# 预测准确率，为了给大家看看
print("在测试集上准确率：", gc.score(X_test, Y_test))

print("在交叉验证当中最好的结果：", gc.best_score_)

print("选择最好的模型是：", gc.best_estimator_)

print("每个超参数每次交叉验证的结果：", gc.cv_results_)

在测试集上准确率： 0.9736842105263158
在交叉验证当中最好的结果： 0.9556661925082978
选择最好的模型是： KNeighborsClassifier(n_neighbors=8)
每个超参数每次交叉验证的结果： {'mean_fit_time': array([0.00065708, 0.00066503, 0.        , 0.00033243, 0.00033331,
       0.        , 0.0008347 , 0.00066487, 0.00099746, 0.00066487]), 'std_fit_time': array([4.64728799e-04, 4.70246478e-04, 0.00000000e+00, 4.70134046e-04,
       4.71370354e-04, 0.00000000e+00, 6.25674223e-04, 4.70134046e-04,
       1.12391596e-07, 4.70134046e-04]), 'mean_score_time': array([0.00166194, 0.00132958, 0.00199469, 0.00099715, 0.00099762,
       0.00199469, 0.00099794, 0.00099738, 0.00099723, 0.00132982]), 'std_score_time': array([4.69797033e-04, 4.70246478e-04, 1.12391596e-07, 1.12391596e-07,
       5.94720425e-07, 6.83651389e-07, 1.07214749e-06, 1.12391596e-07,
       1.12391596e-07, 4.70246478e-04]), 'param_n_neighbors': masked_array(data=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
             mask=[False, False, False, False, False, False, False, False,
                   F